In [2]:
import pandas as pd
import os

In [18]:

sentiment_df = pd.read_csv("data/stocks/agg_sentiment.csv")
sentiment_df["Datetime"] = pd.to_datetime(sentiment_df["timestamp"])
sentiment_df.head()


,ticker,timestamp,sentiment_score,Datetime
0,AAPL,2025-06-16 07:00:00+00:00,0.0,2025-06-16 07:00:00+00:00
1,AAPL,2025-06-16 08:05:00+00:00,0.0,2025-06-16 08:05:00+00:00
2,AAPL,2025-06-16 08:45:00+00:00,0.0,2025-06-16 08:45:00+00:00
3,AAPL,2025-06-16 08:50:00+00:00,0.0,2025-06-16 08:50:00+00:00
4,AAPL,2025-06-16 09:35:00+00:00,0.0,2025-06-16 09:35:00+00:00


In [7]:
import os

stock_folder = "data/stocks/"
tickers = ["AAPL", "TSLA", "NVDA"]

for ticker in tickers:
    path = os.path.join(stock_folder, f"{ticker}_intraday.csv")
    df = pd.read_csv(path)
    print(f"Columns in {ticker}: {df.columns.tolist()}")

Columns in AAPL: ['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume', 'Ticker']
Columns in TSLA: ['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume', 'Ticker']
Columns in NVDA: ['Datetime', 'Close', 'High', 'Low', 'Open', 'Volume', 'Ticker']


In [10]:
stock_folder = "data/stocks/"
from src.Sentiment_Driven_stock_price_movement_predictor.components.data_ingestion import tickers

all_prices = []

for ticker in tickers:
    path = os.path.join(stock_folder, f"{ticker}_intraday.csv")
    df = pd.read_csv(path)
    df["Datetime"] = pd.to_datetime(df["Datetime"])
    df["ticker"] = ticker
    all_prices.append(df)

price_df = pd.concat(all_prices, ignore_index=True)
price_df.head()


,Datetime,Close,High,Low,Open,Volume,Ticker,ticker
0,NaT,AAPL,AAPL,AAPL,AAPL,AAPL,NaN,AAPL
1,2025-06-13 13:30:00+00:00,198.89999389648438,200.3699951171875,198.64500427246094,199.50999450683594,4848028,AAPL,AAPL
2,2025-06-13 13:35:00+00:00,198.27000427246094,199.24000549316406,197.80999755859375,198.88999938964844,1210436,AAPL,AAPL
3,2025-06-13 13:40:00+00:00,197.5,198.2899932861328,197.27000427246094,198.2698974609375,1258742,AAPL,AAPL
4,2025-06-13 13:45:00+00:00,197.38499450683594,197.60000610351562,196.89999389648438,197.52999877929688,1163972,AAPL,AAPL


In [11]:
df.head()

,Datetime,Close,High,Low,Open,Volume,Ticker,ticker
0,NaT,JPM,JPM,JPM,JPM,JPM,NaN,JPM
1,2025-06-13 13:30:00+00:00,264.17999267578125,266.3699951171875,263.6531982421875,265.0799865722656,381684,JPM,JPM
2,2025-06-13 13:35:00+00:00,264.32080078125,265.306396484375,264.1000061035156,264.1700134277344,149741,JPM,JPM
3,2025-06-13 13:40:00+00:00,264.69000244140625,264.81500244140625,263.8999938964844,264.3699951171875,78395,JPM,JPM
4,2025-06-13 13:45:00+00:00,264.43499755859375,264.69000244140625,264.1099853515625,264.67999267578125,71494,JPM,JPM


In [12]:
price_df.head()

,Datetime,Close,High,Low,Open,Volume,Ticker,ticker
0,NaT,AAPL,AAPL,AAPL,AAPL,AAPL,NaN,AAPL
1,2025-06-13 13:30:00+00:00,198.89999389648438,200.3699951171875,198.64500427246094,199.50999450683594,4848028,AAPL,AAPL
2,2025-06-13 13:35:00+00:00,198.27000427246094,199.24000549316406,197.80999755859375,198.88999938964844,1210436,AAPL,AAPL
3,2025-06-13 13:40:00+00:00,197.5,198.2899932861328,197.27000427246094,198.2698974609375,1258742,AAPL,AAPL
4,2025-06-13 13:45:00+00:00,197.38499450683594,197.60000610351562,196.89999389648438,197.52999877929688,1163972,AAPL,AAPL


In [13]:
sentiment_df.head()

,ticker,timestamp,sentiment_score
0,AAPL,2025-06-16 07:00:00+00:00,0.0
1,AAPL,2025-06-16 08:05:00+00:00,0.0
2,AAPL,2025-06-16 08:45:00+00:00,0.0
3,AAPL,2025-06-16 08:50:00+00:00,0.0
4,AAPL,2025-06-16 09:35:00+00:00,0.0


MERGING THE PRICE AND SENTIMENT 

In [23]:
# Load data
price_df = pd.read_csv("data/stocks/AAPL_intraday.csv")
price_df["Datetime"] = pd.to_datetime(price_df["Datetime"])


sentiment_df["Datetime"] = pd.to_datetime(sentiment_df["timestamp"])

# Round sentiment timestamps to 5-minute bins to match prices
sentiment_df["Datetime"] = sentiment_df["Datetime"].dt.floor("5min")

# Aggregate sentiment per 5-min interval
agg_sentiment = sentiment_df.groupby(["Datetime", "ticker"])["sentiment_score"].mean().reset_index()
agg_sentiment.head()

,Datetime,ticker,sentiment_score
0,2025-06-13 10:00:00+00:00,JPM,0.0
1,2025-06-13 13:10:00+00:00,JPM,1.0
2,2025-06-13 14:00:00+00:00,JPM,0.0
3,2025-06-13 14:15:00+00:00,JPM,1.0
4,2025-06-13 14:30:00+00:00,JPM,1.0


In [25]:
agg_sentiment["ticker"]="AAPL"

In [26]:
price_df["ticker"]="AAPL"

In [27]:

# Merge with price
merged_df = pd.merge(price_df, agg_sentiment, on=["Datetime", "ticker"], how="left")
merged_df["sentiment_score"].fillna(0, inplace=True)

/var/folders/zl/csm4ljzx70jgj2wllb_nlz2c0000gn/T/ipykernel_26446/421217875.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df["sentiment_score"].fillna(0, inplace=True)


In [28]:
merged_df.head()

,Datetime,Close,High,Low,Open,Volume,Ticker,ticker,sentiment_score
0,NaT,AAPL,AAPL,AAPL,AAPL,AAPL,NaN,AAPL,0.0
1,2025-06-13 13:30:00+00:00,198.89999389648438,200.3699951171875,198.64500427246094,199.50999450683594,4848028,AAPL,AAPL,0.0
2,2025-06-13 13:35:00+00:00,198.27000427246094,199.24000549316406,197.80999755859375,198.88999938964844,1210436,AAPL,AAPL,0.0
3,2025-06-13 13:40:00+00:00,197.5,198.2899932861328,197.27000427246094,198.2698974609375,1258742,AAPL,AAPL,0.0
4,2025-06-13 13:45:00+00:00,197.38499450683594,197.60000610351562,196.89999389648438,197.52999877929688,1163972,AAPL,AAPL,0.0


In [ ]:

import os
os.makedirs("model_data", exist_ok=True)


In [33]:
for ticker in tickers:
    # ... process and merge logic ...
    merged_df.to_csv(f"model_data/{ticker}_merged_sentiment.csv", index=False)


In [37]:
AAPL_sentiment=pd.read_csv("model_data/AAPL_merged_sentiment.csv")
AAPL_sentiment.head()

,Datetime,Close,High,Low,Open,Volume,Ticker,ticker,sentiment_score
0,2025-06-13 13:30:00+00:00,198.899994,200.369995,198.645004,199.509995,4848028,AAPL,AAPL,0.0
1,2025-06-13 13:35:00+00:00,198.270004,199.240005,197.809998,198.889999,1210436,AAPL,AAPL,0.0
2,2025-06-13 13:40:00+00:00,197.500000,198.289993,197.270004,198.269897,1258742,AAPL,AAPL,0.0
3,2025-06-13 13:45:00+00:00,197.384995,197.600006,196.899994,197.529999,1163972,AAPL,AAPL,0.0
4,2025-06-13 13:50:00+00:00,196.940399,197.660004,196.899994,197.380005,862739,AAPL,AAPL,0.0


DELETING THE FIRST ROW OF ALL SENTIMENT FILES

In [35]:
import os
import pandas as pd

folder_path = "model_data/"  # or "model_data"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        
        df = pd.read_csv(filepath)
        df = df.iloc[1:].reset_index(drop=True)  # Drop first row
        df.to_csv(filepath, index=False)

        print(f"Cleaned: {filename}")


Cleaned: AAPL_merged_sentiment.csv
Cleaned: JPM_merged_sentiment.csv
Cleaned: TSLA_merged_sentiment.csv
Cleaned: MSFT_merged_sentiment.csv
Cleaned: NVDA_merged_sentiment.csv


In [36]:
AAPL_sentiment.head()

,Datetime,Close,High,Low,Open,Volume,Ticker,ticker,sentiment_score
0,2025-06-13 13:30:00+00:00,198.899994,200.369995,198.645004,199.509995,4848028,AAPL,AAPL,0.0
1,2025-06-13 13:35:00+00:00,198.270004,199.240005,197.809998,198.889999,1210436,AAPL,AAPL,0.0
2,2025-06-13 13:40:00+00:00,197.500000,198.289993,197.270004,198.269897,1258742,AAPL,AAPL,0.0
3,2025-06-13 13:45:00+00:00,197.384995,197.600006,196.899994,197.529999,1163972,AAPL,AAPL,0.0
4,2025-06-13 13:50:00+00:00,196.940399,197.660004,196.899994,197.380005,862739,AAPL,AAPL,0.0
